In [3]:
USE_SERVER = True
if USE_SERVER:
    !pip install elasticsearch
    !pip install elasticsearch_dsl
    !pip install pymed
    !pip install gensim
    !pip install torchsummary

  Using cached elasticsearch-7.8.0-py2.py3-none-any.whl (188 kB)
  Using cached elasticsearch_dsl-7.2.1-py2.py3-none-any.whl (53 kB)
  Using cached pymed-0.8.9-py3-none-any.whl (9.6 kB)
  Using cached gensim-3.8.3-cp36-cp36m-manylinux1_x86_64.whl (24.2 MB)
Processing /home/ubuntu/.cache/pip/wheels/a4/9b/d5/85705a7ab783cd6f7bd718f01d3b1396272f30044e3c36401a/smart_open-2.1.0-py3-none-any.whl
  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)


In [115]:
from keras.layers import Dense, Embedding, \
Input, Lambda, RepeatVector, Layer, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers.advanced_activations import ELU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model

from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

In [4]:
from __future__ import print_function
import argparse
import numpy as np

import pandas as pd
import utils
import os
from utils import PROJECT_ROOT, DATA_PATH
import yuval_module.paper_source as PaperSource

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

import py_4.get_mesh_vec as get_mesh_vec
import py_3.sim_matrix_3 as sim_matrix_3
import py_4.get_all_features as get_all_features 


from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import time

from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

In [5]:
FILE = "enriched_labeled_dataset_large" 
if os.path.exists(PROJECT_ROOT + DATA_PATH + FILE):
    print("READING FROM LOCAL")
    if FILE.split(".")[1] == "json":
        df = pd.read_json(PROJECT_ROOT + DATA_PATH + FILE)
    else:
        df = pd.read_csv(PROJECT_ROOT + DATA_PATH + FILE)
    ps = PaperSource()
else:
    print("PULLING FROM S3")
    ps = sim_matrix_3.load_dataset(FILE)
    df = ps.get_dataset()

df.drop(columns=["last_author_country"],inplace=True)
df.rename(columns={'ORG_STATE':'last_author_country'},inplace=True)

print("FILE PULLED")

PULLING FROM S3
FILE PULLED


In [6]:
cd ..

/home/ubuntu/AYP_branch


In [9]:
auth_core = pd.read_csv("data/train_set_author_names.csv")["0"]
auth_eps = pd.read_csv("data/val_set_author_names.csv")["0"]
auth_usecase = pd.read_csv("data/test_set_author_names.csv")["0"]

selection_train = list(set(df['last_author_name']) - set(auth_usecase))[:4000]
selection_test = list(set(df['last_author_name']) - set(auth_usecase))[4000:4500]

df_nonan = df[np.invert(df['mesh'].isna())]

In [10]:
vae_features = get_all_features.VAE_Features(df_nonan)

'/home/ubuntu/AYP/code/models/names_epochs_2_vectorSize_64_window_2.model' already exits. Using existing model to re-generate results.
'/home/ubuntu/AYP/code/models/co_authors_epochs_2_vectorSize_64_window_2.model' already exits. Using existing model to re-generate results.


### TF DATASET

In [50]:
train_examples = vae_features.get_all_features(df_nonan[df_nonan['last_author_name'].isin(selection_train)])

X_train = tf.convert_to_tensor(train_examples)

train_dataset = tf.data.Dataset.from_tensor_slices(train_examples)

/home/ubuntu/AYP_branch/code/py_4/get_all_features.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['co_authors']=df.authors.apply( lambda x: [i['name'] for i in x] )


Using old scaler


In [86]:
int(X_train.shape[0])

19930

In [126]:
def sent_generator(X_train, batchsize):
    # Create iterator that reads dataset file batch by batch 
    #your code here
    #Suffle the data file
#     indexes = np.random.choice (range(len(full)),len(full),False)
    
#     shuffled = full.iloc[indexes]

    reader = [X_train[batchsize*i: batchsize*i +batchsize,:] for i in range(int(np.ceil(int(X_train.shape[0])//batchsize)))]

    
    for df in reader:
        yield [tf.cast(df,dtype = tf.float32), tf.cast(df,dtype = tf.float32)] 
        
print([a for a in sent_generator(train_examples,  32)][:1])

[[<tf.Tensor 'Cast_15:0' shape=(32, 64) dtype=float32>, <tf.Tensor 'Cast_16:0' shape=(32, 64) dtype=float32>]]


### TF MODEL

In [116]:
num_sample = train_examples.shape[0]
input_dim = train_examples.shape[1]
hidden_dim = int(np.sqrt(input_dim))
n_z = 8

n_epoch = 100
m = 64

In [117]:
print("The number of samples: ", num_sample)
print("The input dim: ", input_dim)

The number of samples:  19930
The input dim:  64


In [118]:
inputs = Input(shape=(input_dim,))
h_q = Dense(hidden_dim, activation='relu')(inputs)
mu = Dense(n_z, activation='linear')(h_q)
log_sigma = Dense(n_z, activation='linear')(h_q)

In [119]:
def sample_z(args):
    mu, log_sigma = args
    eps = K.random_normal(shape=(m, n_z), mean=0., stddev=1.)
    return mu + K.exp(log_sigma / 2) * eps


# Sample z ~ Q(z|X)
z = Lambda(sample_z)([mu, log_sigma])

In [120]:
# P(X|z) -- decoder
decoder_hidden = Dense(hidden_dim, activation='relu')
decoder_out = Dense(input_dim, activation='linear')

h_p = decoder_hidden(z)
outputs = decoder_out(h_p)

In [121]:
def zero_loss(y_true, y_pred):
    # Return tensor filled with ones with shape equal generated sequence shape
    return K.zeros_like(y_pred)

In [122]:
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)
        # Create tensor (batch_size, max_sequence_len) filled with ones to consider all elements of generated sequence 
        self.target_weights = tf.constant(np.ones((batch_size, input_dim)), tf.float32)

    def vae_loss(self, x, x_decoded_mean):
        # Compute sequence reconstruction loss
        recon = tf.keras.losses.MSE(x,x_decoded_mean)
        # Compute KL-divergence as Variational loss 
        kl_loss = -0.5 * K.sum(1 + log_sigma - K.square(mu) - K.exp(log_sigma), axis=-1)
        # Composite loss (reconstruction loss + Variational loss)
        return recon + kl_loss

    def call(self, inputs):
        x = inputs[0] # input sequence
        x_decoded_mean = inputs[1] # reconstructed sequence
        print(x.shape, x_decoded_mean.shape)
        loss = self.vae_loss(x, x_decoded_mean) # Compute loss of the model
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape
        return K.ones_like(x)

In [123]:
# Create custom layer for loss computing
loss_layer = CustomVariationalLayer()([inputs, outputs])

vae = Model(inputs, [loss_layer])
# Use Adam optimizer with learning rate = 0.01
opt = Adam(lr=0.01)
vae.compile(optimizer=opt, loss=[zero_loss])
# Show model structure
vae.summary()

(?, 64) (64, 64)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
dense_17 (Dense)                (None, 8)            520         input_6[0][0]                    
__________________________________________________________________________________________________
dense_18 (Dense)                (None, 8)            72          dense_17[0][0]                   
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 8)            72          dense_17[0][0]                   
____________________________________________________________________________________________

In [124]:
def create_model_checkpoint(model_name):
    filepath = f"models/{model_name}.h5"
    directory = os.path.dirname(filepath)
    try:
        # Check if directory exists
        os.stat(directory)
    except:
        # If directory doesn't exist, create the directory
        os.mkdir(directory)
    # Save model states
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=False)
    return checkpointer

# Create model checkpointer
checkpointer = create_model_checkpoint('vae_seq2seq')

In [130]:
nb_epoch = 25 # number of epochs for model training
n_steps = X_train.shape[0] // batch_size # Number of steps per epoch
for counter in range(nb_epoch):
    print('-------epoch: ', counter, '-------')
    # Train model. Test and save model after every epoch
    vae.fit_generator(sent_generator(X_train, batch_size),
                      steps_per_epoch=n_steps, epochs=1, callbacks=[checkpointer])
vae.save(r'vae_lstmFull32dim96hid.h5')

-------epoch:  0 -------
Epoch 1/1


TypeError: len is not well defined for symbolic Tensors. (strided_slice_2488:0) Please call `x.shape` rather than `len(x)` for shape information.

### OTHER OPTION

In [76]:
?tf.cast

Signature: tf.cast(x, dtype, name=None)
Docstring:
Casts a tensor to a new type.

The operation casts `x` (in case of `Tensor`) or `x.values`
(in case of `SparseTensor` or `IndexedSlices`) to `dtype`.

For example:

```python
x = tf.constant([1.8, 2.2], dtype=tf.float32)
tf.dtypes.cast(x, tf.int32)  # [1, 2], dtype=tf.int32
```

The operation supports data types (for `x` and `dtype`) of
`uint8`, `uint16`, `uint32`, `uint64`, `int8`, `int16`, `int32`, `int64`,
`float16`, `float32`, `float64`, `complex64`, `complex128`, `bfloat16`.
In case of casting from complex types (`complex64`, `complex128`) to real
types, only the real part of `x` is returned. In case of casting from real
types to complex types (`complex64`, `complex128`), the imaginary part of the
returned value is set to `0`. The handling of complex types here matches the
behavior of numpy.

Args:
  x: A `Tensor` or `SparseTensor` or `IndexedSlices` of numeric type. It could
    be `uint8`, `uint16`, `uint32`, `uint64`, `int8`, `

In [72]:
X_train = tf.convert_to_tensor(train_examples, dtype = tf.float32)
train_dataset = tf.data.Dataset.from_tensor_slices(train_examples)
iter_ds = train_dataset.make_initializable_iterator() 

In [80]:
import tensorflow as tf
import numpy as np
import os


mb_size = 64
z_dim = 8
X_dim = y_dim = X_train.shape[1]
h_dim = 32
c = 0
lr = 1e-3



def xavier_init(size):
    in_dim = 64.0
    xavier_stddev = 1. / tf.sqrt(in_dim // 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


# =============================== Q(z|X) ======================================

X = tf.placeholder(tf.float32, shape=[None, X_dim])
z = tf.placeholder(tf.float32, shape=[None, z_dim])

Q_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

Q_W2_mu = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_mu = tf.Variable(tf.zeros(shape=[z_dim]))

Q_W2_sigma = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_sigma = tf.Variable(tf.zeros(shape=[z_dim]))


def Q(X):
    h = tf.nn.relu(tf.matmul(X, Q_W1) + Q_b1)
    z_mu = tf.matmul(h, Q_W2_mu) + Q_b2_mu
    z_logvar = tf.matmul(h, Q_W2_sigma) + Q_b2_sigma
    return z_mu, z_logvar


def sample_z(mu, log_var):
    eps = tf.random_normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var // 2) * eps


# =============================== P(X|z) ======================================

P_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
P_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

P_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
P_b2 = tf.Variable(tf.zeros(shape=[X_dim]))


def P(z):
    h = tf.nn.relu(tf.matmul(z, P_W1) + P_b1)
    logits = tf.matmul(h, P_W2) + P_b2
    return logits


# =============================== TRAINING ====================================

z_mu, z_logvar = Q(X)
z_sample = sample_z(z_mu, z_logvar)
_, logits = P(z_sample)

# Sampling from random z
X_samples, _ = P(z)

# E[log P(X|z)]
recon_loss = tf.keras.losses.MSE(X,logits)

# D_KL(Q(z|X) || P(z)); calculate in closed form as both dist. are Gaussian
kl_loss = 0.5 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)
# VAE loss
vae_loss = tf.reduce_mean(recon_loss + kl_loss)

solver = tf.train.AdamOptimizer().minimize(vae_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

for it in range(1000000):
    X_mb, _ = iter_ds.get_next(mb_size)

    _, loss = sess.run([solver, vae_loss], feed_dict={X: X_mb})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('Loss: {:.4}'. format(loss))
        print()

OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.